In [4]:
import numpy as np
import qililab as ql

## ugly solution because we have not created a proper package
import sys

sys.path.insert(1, "../utils")

from qst_qpt_helper_functions import *

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

import os

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit


# api = API(ConnectionConfiguration(username="qat", api_key="meow"))
api = API(ConnectionConfiguration(username="apalacios", api_key="3ec51562-3ff2-4f7b-add8-b21e1645a89d"))

api.select_device_id(9)

Is setting the logger perhaps a bit ugly for remote users to have to do manually? I have not seen any experiment where it is preferred to not be set to a higher level

In [5]:
ql.logger.setLevel(40)  # Set qililab's logger to a higher level so it only shows error messages

In [7]:
# Amplitude set in Qblox terms (0.5 times mV)
Time_START = 0
Time_STOP = 200000
Time_STEP = 20  ## this had two more zeros, but with the software loop if was taking 7more than 7 minutes to send.

SQUARE_duration = 4000
GAUSS_duration = 2000

NSHOTS = 2000

QUBIT = 4

In [8]:
# The shape of the pulses is introduced here
gauss_wf = ql.Gaussian(amplitude=1, duration=GAUSS_duration, num_sigmas=4.0)  # Drag pulse executing pi rotation
square_wf = ql.Square(amplitude=1, duration=SQUARE_duration)  # square wavefront (readout pulse)

weights_shape = ql.Square(amplitude=1, duration=SQUARE_duration)  # This shape implies that the weights are uniform
## this uniform weighting has nothing to do with the thresholding, it's because Qblox allows for weighting the points
## of the waveform that you sent differently. This feature in particular is designed for aiding in making the square
## pulses as square as possible (since one has to correct the edges for a digital square signal). We set it to be uniform
## always, but I believe we are making this correction elsewhere. Check with Guille though.

In [ ]:
qp_multiple_measurement = ql.QProgram()
QUBIT = 4

# Defining the variable Frequency for the driving pulse
wait_time = qp_multiple_measurement.variable(
    ql.Domain.Time
)  # Possible domains: Scalar, Time, Frequency, Phase and Voltage

# Averaging over NSHOTS iterations
with qp_multiple_measurement.average(NSHOTS):
    # Loop over all wait times
    with qp_multiple_measurement.for_loop(variable=wait_time, start=Time_START, stop=Time_STOP, step=Time_STEP):
        qp_multiple_measurement.play(bus=f"drive_q{QUBIT}_bus", waveform=gauss_wf)  # execute the pi pulse
        qp_multiple_measurement.wait(bus=f"drive_q{QUBIT}_bus", duration=wait_time)  # wait

        # qp_t1.sync()  # so that the next bus (for the readout) executes after the previous (drive)
        ## a sync must be added between different buses if we want them to be consecutive. Each bus
        ## refers to either a particular qubit's sequence of DRAGs, a particular qubit's readout pulse
        ## or a particular qubit pair's sequence of two-qubit gates

        # READOUT PULSE
        ## workaround to avoid dynamic syncing
        wait_time_before_readout = GAUSS_duration + wait_time
        qp_multiple_measurement.wait(bus=f"readout_q{QUBIT}_bus", duration=wait_time_before_readout)

        qp_multiple_measurement.play(
            bus=f"readout_q{QUBIT}_bus", waveform=square_wf, wait_time=4
        )  ## execute readout pulse
        qp_multiple_measurement.acquire(
            bus=f"readout_q{QUBIT}_bus", weights=ql.IQPair(I=weights_shape, Q=weights_shape)
        )  # Collect data from readout

Exemplary code snippet for parallel loops

In [33]:
from qililab.qprogram.blocks import ForLoop

qp_t1 = ql.QProgram()
wait_time = qp_t1.variable(ql.Domain.Time)
wait_time_RO = qp_t1.variable(ql.Domain.Time)
Time_START_RO = Time_START + GAUSS_duration
Time_STOP_RO = Time_STOP + GAUSS_duration

with qp_t1.average(NSHOTS):
    with qp_t1.parallel(
        loops=[
            ForLoop(variable=wait_time, start=Time_START, stop=Time_STOP, step=Time_STEP),
            ForLoop(variable=wait_time_RO, start=Time_START_RO, stop=Time_STOP_RO, step=Time_STEP),
        ]
    ):
        qp_t1.play(bus=f"drive_q{QUBIT}_bus", waveform=gauss_wf)  # execute the pi pulse
        qp_t1.wait(bus=f"drive_q{QUBIT}_bus", duration=wait_time)  # wait

        qp_t1.wait(bus=f"readout_q{QUBIT}_bus", duration=wait_time_RO)

        qp_t1.play(bus=f"readout_q{QUBIT}_bus", waveform=square_wf, wait_time=4)  ## execute readout pulse
        qp_t1.acquire(
            bus=f"readout_q{QUBIT}_bus", weights=ql.IQPair(I=weights_shape, Q=weights_shape)
        )  # Collect data from readout

full_qprogram = qp_t1.to_dict()

In [28]:
# Building the Qprogram
qp_t1 = ql.QProgram()

# Defining the variable Frequency for the driving pulse
wait_time = qp_t1.variable(ql.Domain.Time)  # Possible domains: Scalar, Time, Frequency, Phase and Voltage

# Averaging over NSHOTS iterations
with qp_t1.average(NSHOTS):
    # Loop over all wait times
    with qp_t1.for_loop(variable=wait_time, start=Time_START, stop=Time_STOP, step=Time_STEP):
        qp_t1.play(bus=f"drive_q{QUBIT}_bus", waveform=gauss_wf)  # execute the pi pulse
        qp_t1.wait(bus=f"drive_q{QUBIT}_bus", duration=wait_time)  # wait

        # qp_t1.sync()  # so that the next bus (for the readout) executes after the previous (drive)
        ## a sync must be added between different buses if we want them to be consecutive. Each bus
        ## refers to either a particular qubit's sequence of DRAGs, a particular qubit's readout pulse
        ## or a particular qubit pair's sequence of two-qubit gates

        # READOUT PULSE
        ## workaround to avoid dynamic syncing
        wait_time_before_readout = GAUSS_duration + wait_time
        qp_t1.wait(bus=f"readout_q{QUBIT}_bus", duration=wait_time_before_readout)

        qp_t1.play(bus=f"readout_q{QUBIT}_bus", waveform=square_wf, wait_time=4)  ## execute readout pulse
        qp_t1.acquire(
            bus=f"readout_q{QUBIT}_bus", weights=ql.IQPair(I=weights_shape, Q=weights_shape)
        )  # Collect data from readout

full_qprogram = qp_t1.to_dict()

# This code can't work yet because the sync after a wait time is not properly implemented yet.

In [29]:
Wait_duration_list = np.linspace(Time_START, Time_STOP, Time_STEP)

In [34]:
result_id = api.execute(qprogram=full_qprogram)[0]
print(result_id)

{
  "title": "Unauthorized",
  "status": 401,
  "detail": "JWTExpired: Error verifying the authorisation access token. Expired at 1709546789, time: 1709567069(leeway: 60) 401 Client Error:  for url: https://qilimanjaroqaas.ddns.net:8080/api/v1/circuits"
}
{"title":"Unauthorized","status":401,"detail":"JWTExpired: Error verifying the authorisation access token. Expired at 1709546789, time: 1709567069(leeway: 60)"}



9602


### Post-processing of data is now going to the same function as if it were coming from a circuit, but the post-processing will likely have to be different.

In [44]:
results = api.get_result(result_id)

{
  "title": "Unauthorized",
  "status": 401,
  "detail": "JWTExpired: Error verifying the authorisation access token. Expired at 1709567370, time: 1709567508(leeway: 60) 401 Client Error:  for url: https://qilimanjaroqaas.ddns.net:8080/api/v1/jobs/9602"
}
{"title":"Unauthorized","status":401,"detail":"JWTExpired: Error verifying the authorisation access token. Expired at 1709567370, time: 1709567508(leeway: 60)"}

Your job with id 9602 failed.


In [45]:
print(api.get_job(result_id).logs)

Your job with id 9602 failed.


Interruption in stage='execute': Dynamic syncing is not implemented yet. (<class 'NotImplementedError'>). Traceback:
  File "/home/qili-docker/qgqs/services/device_service/src/utils/decorators.py", line 20, in wrapper
    return func(*args, **kwargs)

  File "/home/qili-docker/qgqs/services/device_service/src/api/slurm_executions.py", line 77, in _execute
    return execution_class.execute_qprogram(

  File "/home/qili-docker/qgqs/services/device_service/src/service/executions/qililab_executions.py", line 76, in execute_qprogram
    result = platform.execute_qprogram(qprogram=qprogram)

  File "/home/qili-docker/miniconda3/envs/submitit/lib/python3.10/site-packages/qililab/platform/platform.py", line 621, in execute_qprogram
    return self._execute_qprogram_with_qblox(qprogram=qprogram, bus_mapping=bus_mapping, debug=debug)

  File "/home/qili-docker/miniconda3/envs/submitit/lib/python3.10/site-packages/qililab/platform/platform.py", line 638, in _execute_qprogram_with_qblox
    seque

In [ ]:
data_probabilities = process_returned_dataformat(results, nqubits=1)

In [ ]:
probabilities_excited = data_probabilities[:, 1]

fit_up2 = -1
coeffs_fit = np.polyfit(Wait_duration_list[:fit_up2], np.log10(probabilities_excited[:fit_up2]), 1)
poly1d_fn = np.poly1d(coeffs_fit)  # poly1d_fn is now a function which takes in x and returns an estimate for y
exponent = coeffs_fit[0]
T1 = -1 / exponent

# Plot the data and the fitted function
plt.figure(figsize=(8, 6))
plt.plot(Wait_duration_list, np.log10(probabilities_excited), label="data")
plt.plot(Wait_duration_list, poly1d_fn(Wait_duration_list), "r", label=f"Fitted T1={T1/1000:.3f} us")
plt.legend()
plt.xlabel("Wait time (ns)")
plt.ylabel("Prob($|1\\rangle$)")